In [53]:
import numpy as np

In [54]:
def find_one_dimensional_route(start, end):
    vertical_diff, horizontal_diff = end - start
    if vertical_diff == 0:
        if horizontal_diff < 0:
            return "<" * abs(horizontal_diff)
        else:
            return ">" * abs(horizontal_diff)
    if horizontal_diff == 0:
        if vertical_diff < 0:
            return "^" * abs(vertical_diff)
        else:
            return "v" * abs(vertical_diff)
    raise ValueError("Not 1 dimensional!")


def find_shortest_routes(start, end, available):
    available = [tuple(field) for field in available]
    shortest_routes = []
    vertical_diff, horizontal_diff = end - start
    if (vertical_diff == 0) or (horizontal_diff == 0):
        return [find_one_dimensional_route(start, end) + "A"]

    corners = [start + (0, horizontal_diff), start + (vertical_diff, 0)]
    corners = [corner for corner in corners]
    for corner in corners:

        if tuple(corner) in available:
            route = (
                find_one_dimensional_route(start, corner)
                + find_one_dimensional_route(corner, end)
                + "A"
            )
            shortest_routes.append(route)
    return shortest_routes


def find_shortest_routes_between_buttons(first_place, second_place, pad):
    if pad == "numeric":
        loc = {
            "A": np.array([3, 2]),
            "0": np.array([3, 1]),
            "1": np.array([2, 0]),
            "2": np.array([2, 1]),
            "3": np.array([2, 2]),
            "4": np.array([1, 0]),
            "5": np.array([1, 1]),
            "6": np.array([1, 2]),
            "7": np.array([0, 0]),
            "8": np.array([0, 1]),
            "9": np.array([0, 2]),
        }
    if pad == "directional":
        loc = {
            "A": np.array([0, 2]),
            "^": np.array([0, 1]),
            "<": np.array([1, 0]),
            "v": np.array([1, 1]),
            ">": np.array([1, 2]),
        }

    first_loc = loc[first_place]
    second_loc = loc[second_place]

    routes = find_shortest_routes(first_loc, second_loc, loc.values())
    return routes


find_shortest_routes_between_buttons("A", "4", "numeric")
find_shortest_routes_between_buttons("3", "4", "numeric")

['<<^A', '^<<A']

In [55]:
# def find_shortest_route_numeric_pad(first_place, second_place):
#     loc = {"A": np.array([3, 2]), "0": np.array([3, 1]), "1": np.array([2, 0]), "2": np.array([2, 1]),
#            "3": np.array([2, 2]), "4": np.array([1, 0]), "5": np.array([1, 1]), "6": np.array([1, 2]),
#            "7": np.array([0, 0]), "8": np.array([0, 1]), "9": np.array([0, 2])}
#     first_loc = loc[first_place]
#     second_loc = loc[second_place]

#     vertical_diff, horizontal_diff = second_loc - first_loc

#     route = ""
#     if vertical_diff <= 0:
#         route += "^" * abs(vertical_diff)
#         if horizontal_diff <= 0:
#             route += "<" *abs(horizontal_diff)
#         else:
#             route += ">" * horizontal_diff
#     else:
#         if horizontal_diff <= 0:
#             route += "<" *abs(horizontal_diff)
#         else:
#             route += ">" * horizontal_diff
#         route += "v" * vertical_diff

#     return route + "A"


# def find_shortest_route_directional_pad(first_place, second_place):
#     loc = {"A": np.array([0, 2]), "^": np.array([0, 1]), "<": np.array([1, 0]), "v": np.array([1, 1]),
#            ">": np.array([1, 2])}
#     first_loc = loc[first_place]
#     second_loc = loc[second_place]

#     vertical_diff, horizontal_diff = second_loc - first_loc

#     route = ""

#     # The gap is somewhere else, so horizontal/vertical takes different precedence
#     if vertical_diff >= 0:
#         route += "v" * abs(vertical_diff)
#         if horizontal_diff <= 0:
#             route += "<" *abs(horizontal_diff)
#         else:
#             route += ">" * horizontal_diff
#     else:
#         if horizontal_diff <= 0:
#             route += "<" *abs(horizontal_diff)
#         else:
#             route += ">" * horizontal_diff
#         route += "^" * abs(vertical_diff)
#     return route + "A"


def create_all_shortest_routes_numeric():
    shortest_route = {}
    for first_place in "A1234567890":
        for second_place in "A1234567890":
            if first_place == second_place:
                continue
            shortest_route[first_place + second_place] = (
                find_shortest_routes_between_buttons(
                    first_place, second_place, "numeric"
                )
            )
    return shortest_route


def create_all_shortest_routes_directional():
    shortest_route = {}
    for first_place in "A<>^v":
        for second_place in "A<>^v":
            shortest_route[first_place + second_place] = (
                find_shortest_routes_between_buttons(
                    first_place, second_place, "directional"
                )
            )
    return shortest_route


sequence_of_button_pushes_numeric = create_all_shortest_routes_numeric()
sequence_of_button_pushes_directional = create_all_shortest_routes_directional()

In [35]:
import itertools

In [92]:
def find_button_combinations_shortest(code, shortest_route: dict):
    button_pushes = []
    for current, next in zip(code[:-1], code[1:]):
        button_pushes.append(shortest_route[current + next])
    return ["".join(option) for option in itertools.product(*button_pushes)]


def find_button_combinations_codes_shortest(codes, shortest_route: dict):
    result = []
    for code in codes:
        result.extend(find_button_combinations_shortest(code, shortest_route))
    return result


def find_min_len_buttons_second_bot(code):
    numeric_bot = find_button_combinations_shortest(
        code, sequence_of_button_pushes_numeric
    )
    new_codes = ["A" + option for option in numeric_bot]
    first_directional_bot = find_button_combinations_codes_shortest(
        new_codes, sequence_of_button_pushes_directional
    )

    new_codes = ["A" + option for option in first_directional_bot]
    second_direction_bot = find_button_combinations_codes_shortest(
        first_directional_bot, sequence_of_button_pushes_directional
    )
    return min(len(buttons) for buttons in second_direction_bot)


find_min_len_buttons_second_bot("37")

20

In [36]:
def find_button_combinations(code, shortest_route: dict):
    starting_point = "A"
    code = starting_point + code
    button_pushes = []
    for current, next in zip(code[:-1], code[1:]):
        button_pushes.append(shortest_route[current + next])
    return ["".join(option) for option in itertools.product(*button_pushes)]


def find_button_combinations_codes(codes, shortest_route: dict):
    result = []
    for code in codes:
        result.extend(find_button_combinations(code, shortest_route))
    return result


code = "029A"
numeric_bot = find_button_combinations(code, sequence_of_button_pushes_numeric)
first_directional_bot = find_button_combinations_codes(
    numeric_bot, sequence_of_button_pushes_directional
)
second_direction_bot = find_button_combinations_codes(
    first_directional_bot, sequence_of_button_pushes_directional
)
min(len(buttons) for buttons in second_direction_bot)


def find_min_len_buttons_second_bot(code):
    numeric_bot = find_button_combinations(code, sequence_of_button_pushes_numeric)
    first_directional_bot = find_button_combinations_codes(
        numeric_bot, sequence_of_button_pushes_directional
    )
    second_direction_bot = find_button_combinations_codes(
        first_directional_bot, sequence_of_button_pushes_directional
    )
    return min(len(buttons) for buttons in second_direction_bot)

In [37]:
def calculate_complexity(code):
    len_buttons = find_min_len_buttons_second_bot(code)
    complexity = len_buttons * int(code[:-1])
    # print(f"{len_buttons} * {int(code[:-1])}")
    return complexity


codes = ["279A", "341A", "459A", "540A", "085A"]
sum(calculate_complexity(code) for code in codes)

123096

In [ ]:
numeric_bot = find_button_combinations("379A", sequence_of_button_pushes_numeric)
first_directional_bot = find_button_combinations_codes(
    numeric_bot, sequence_of_button_pushes_directional
)
first_directional_bot
find_button_combinations_codes(
    first_directional_bot, sequence_of_button_pushes_directional
)

['v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>Av<<A>A>^AAAvA<^A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>Av<<A>A>^AAAvA^<A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>Av<<A>A^>AAAvA<^A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>Av<<A>A^>AAAvA^<A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA^>AA<A>Av<<A>A>^AAAvA<^A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA^>AA<A>Av<<A>A>^AAAvA^<A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA^>AA<A>Av<<A>A^>AAAvA<^A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA^>AA<A>Av<<A>A^>AAAvA^<A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^Av<A>^AA<A>Av<<A>A>^AAAvA<^A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^Av<A>^AA<A>Av<<A>A>^AAAvA^<A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^Av<A>^AA<A>Av<<A>A^>AAAvA<^A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^Av<A>^AA<A>Av<<A>A^>AAAvA^<A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^Av<A^>AA<A>Av<<A>A>^AAAvA<^A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^Av<A^>AA<A>Av<<A>A>^AAAvA^<A>A',
 'v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AAv

In [62]:
buttons_directional_pad = {
    (0, 2): "A",
    (0, 1): "^",
    (1, 0): "<",
    (1, 1): "v",
    (1, 2): ">",
}
buttons_numerical_pad = {
    (0, 0): "7",
    (0, 1): "8",
    (0, 2): "9",
    (1, 0): "4",
    (1, 1): "5",
    (1, 2): "6",
    (2, 0): "1",
    (2, 1): "2",
    (2, 2): "3",
    (3, 1): "0",
    (3, 2): "A",
}

In [64]:
def find_button_presses(code, start_location):
    directions = {
        "^": np.array([-1, 0]),
        ">": np.array([0, 1]),
        "v": np.array([1, 0]),
        "<": np.array([0, -1]),
    }
    output = []
    current_location = start_location
    for move in code:
        if move == "A":
            output.append(current_location)
        else:
            current_location = tuple(current_location + directions[move])
    return output


def run_code(code, mode):
    if mode == "directional":
        button_locations = buttons_directional_pad
        start_position = (0, 2)
    elif mode == "numeric":
        button_locations = buttons_numerical_pad
        start_position = (3, 2)
    pressed_buttons = find_button_presses(code, start_position)
    return "".join(button_locations[location] for location in pressed_buttons)


total_string = "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"
directional = run_code(total_string[12:35], "directional")
directional2 = run_code(directional, "directional")
numeric = run_code(directional2, "numeric")
numeric

'4'

In [80]:
print(total_string[12:22])
print(total_string[22:23])
print(total_string[23:33])

<vA<AA>>^A
A
vA<^A>AAvA


In [81]:
print(directional)

v<<AA>^AA>A


In [76]:
run_code("<vA<AA>>^A", "directional")

'v<<A'

In [77]:
len(total_string[12:22])

10

In [66]:
directional

'v<<AA>^AA>A'

In [78]:
len("v<A<AA>>^A")

10

In [67]:
directional2

'<<^^A'

In [68]:
numeric

'4'

In [65]:
total_string[12:35]

'<vA<AA>>^AAvA<^A>AAvA^A'

In [ ]:
code = "379A"
numeric_bot = find_shortest_route_numeric_code(code)
first_directional_bot = find_shortest_route_directional_code(numeric_bot)
second_direction_bot = find_shortest_route_directional_code(first_directional_bot)
second_direction_bot
len(second_direction_bot)

# Part 2

In [96]:
from collections import Counter

In [181]:
def get_transitions(string):
    return [string[i] + string[i + 1] for i in range(len(string) - 1)]


def get_transitions_one_layer_up(sequence_of_buttons):
    transitions_layer_up = {}
    for transition, buttons in sequence_of_buttons.items():
        transition_result = []
        for possibility in buttons:
            transitions = get_transitions(possibility)
            transition_result.append(Counter(transitions))
        transitions_layer_up[transition] = transition_result
    return transitions_layer_up


up_transitions_numeric = get_transitions_one_layer_up(sequence_of_button_pushes_numeric)


def get_transitions_one_layer_up(sequence_of_buttons):
    transitions_layer_up = {}
    for transition, buttons in sequence_of_buttons.items():
        transition_result = []
        for possibility in buttons:
            transitions = get_transitions(possibility)
            transition_result.append((Counter(transitions), possibility[0]))
        transitions_layer_up[transition] = transition_result
    return transitions_layer_up


up_transitions_directional = get_transitions_one_layer_up(
    sequence_of_button_pushes_directional
)

In [182]:
import functools

In [218]:
@functools.cache
def n_buttons_n_directionals_up(transition, levels_up):
    """Only for directional pad"""
    if levels_up == 1:
        # All these lengths are equal by definition
        return len(sequence_of_button_pushes_directional[transition][0])
    possible = []
    options = up_transitions_directional[transition]
    for option, first_move in options:
        this_score = n_buttons_n_directionals_up("A" + first_move, levels_up - 1)
        for trans, n in option.items():
            this_score += n * n_buttons_n_directionals_up(trans, levels_up - 1)
        this_score += sum(option.values())  # The number of times A needs to be pressed
        possible.append(this_score)
    return min(possible)


def buttons_n_machines_up(code, n_machines):
    numeric_bot = find_button_combinations(code, sequence_of_button_pushes_numeric)
    transition_options = [
        (Counter(get_transitions(option)), option[0]) for option in numeric_bot
    ]

    possible = []
    for option, first_move in transition_options:
        this_score = n_buttons_n_directionals_up("A" + first_move, n_machines)
        for trans, n in option.items():
            this_score += n * n_buttons_n_directionals_up(trans, n_machines)
        this_score += sum(option.values())  # The number of times A needs to be pressed
        possible.append(this_score)
    return min(possible)


# n_buttons_n_directionals_up("A<", 2)
buttons_n_machines_up("3A", 1)

1 ^A 2
1 Av 3
1 vA 3


13

In [219]:
def find_button_combinations_shortest(code, shortest_route: dict):
    button_pushes = []
    for current, next in zip(code[:-1], code[1:]):
        button_pushes.append(shortest_route[current + next])
    return ["".join(option) for option in itertools.product(*button_pushes)]


def find_button_combinations_codes_shortest(codes, shortest_route: dict):
    result = []
    for code in codes:
        result.extend(find_button_combinations_shortest(code, shortest_route))
    return result


def find_min_len_buttons_second_bot(code):
    numeric_bot = find_button_combinations_shortest(
        code, sequence_of_button_pushes_numeric
    )
    new_codes = ["A" + option for option in numeric_bot]
    first_directional_bot = find_button_combinations_codes_shortest(
        new_codes, sequence_of_button_pushes_directional
    )

    new_codes = ["A" + option for option in first_directional_bot]
    second_direction_bot = find_button_combinations_codes_shortest(
        first_directional_bot, sequence_of_button_pushes_directional
    )
    return min(len(buttons) for buttons in second_direction_bot)


find_min_len_buttons_second_bot("3A")

12

In [150]:
[
    len(x)
    for x in find_button_combinations_codes(
        ["v<<A"], sequence_of_button_pushes_directional
    )
]

[10, 10]

In [145]:
sequence_of_button_pushes_directional["A<"]

['v<<A']

In [142]:
up_transitions_directional["<>"]

[Counter({'>>': 1, '>A': 1})]

In [ ]:
result = {}
for transition, buttons in sequence_of_button_pushes_directional.items():
    # print(transition, buttons)
    transition_result = []
    for possibility in buttons:
        transitions = [
            possibility[i] + possibility[i + 1] for i in range(len(possibility) - 1)
        ]
        transition_result.append(Counter(transitions))
    result[transition] = transition_result

In [18]:
def find_min_len_buttons_second_bot(code):
    numeric_bot = find_button_combinations(code, sequence_of_button_pushes_numeric)
    first_directional_bot = find_button_combinations_codes(
        numeric_bot, sequence_of_button_pushes_directional
    )
    second_direction_bot = find_button_combinations_codes(
        first_directional_bot, sequence_of_button_pushes_directional
    )
    return min(len(buttons) for buttons in second_direction_bot)


find_min_len_buttons_second_bot("379A")

NameError: name 'find_button_combinations' is not defined

In [24]:
def find_state_addition(first_place, second_place, pad):
    if pad == "numeric":
        loc = {
            "A": np.array([3, 2]),
            "0": np.array([3, 1]),
            "1": np.array([2, 0]),
            "2": np.array([2, 1]),
            "3": np.array([2, 2]),
            "4": np.array([1, 0]),
            "5": np.array([1, 1]),
            "6": np.array([1, 2]),
            "7": np.array([0, 0]),
            "8": np.array([0, 1]),
            "9": np.array([0, 2]),
        }
    if pad == "directional":
        loc = {
            "A": np.array([0, 2]),
            "^": np.array([0, 1]),
            "<": np.array([1, 0]),
            "v": np.array([1, 1]),
            ">": np.array([1, 2]),
        }

    first_loc = loc[first_place]
    second_loc = loc[second_place]

    routes = find_state_addition_location(first_loc, second_loc, loc.values())
    return routes


def find_state_addition_location(start, end, available):
    manhattan_distance = np.sum(np.abs(end - start))

    possible_end_states = ""
    vertical_diff, horizontal_diff = end - start

    available = [tuple(field) for field in available]
    corners = [tuple(start + (0, horizontal_diff)), tuple(start + (vertical_diff, 0))]
    corners = [corner for corner in corners if corner in available]

    if horizontal_diff < 0 and tuple(end - (0, horizontal_diff)) in corners:
        possible_end_states += "<"
    if horizontal_diff > 0 and tuple(end - (0, horizontal_diff)) in corners:
        possible_end_states += ">"
    if vertical_diff < 0 and tuple(end - (vertical_diff, 0)) in corners:
        possible_end_states += "^"
    if vertical_diff > 0 and tuple(end - (vertical_diff, 0)) in corners:
        possible_end_states += "v"
    possible_end_states = possible_end_states or "*"  # "*" means: stay where u are

    return list(itertools.product(possible_end_states, [manhattan_distance]))


find_state_addition("^", "<", "directional")

[('<', 2)]

In [30]:
def find_all_state_transitions_numeric():
    shortest_route = {}
    for first_place in "A1234567890":
        for second_place in "A1234567890":
            shortest_route[first_place + second_place] = find_state_addition(
                first_place, second_place, "numeric"
            )
    return shortest_route


def find_all_state_transitions_directional():
    shortest_route = {}
    for first_place in "A<>^v":
        for second_place in "A<>^v":
            shortest_route[first_place + second_place] = find_state_addition(
                first_place, second_place, "directional"
            )
    return shortest_route


state_transitions_numeric = find_all_state_transitions_numeric()
state_transitions_directional = find_all_state_transitions_directional()

In [ ]:
def find_button_combinations(code, shortest_route: dict):
    starting_point = "A"
    code = starting_point + code
    button_pushes = []
    for current, next in zip(code[:-1], code[1:]):
        button_pushes.append(shortest_route[current + next])
    return ["".join(option) for option in itertools.product(*button_pushes)]

In [ ]:
code = "029A"


def find_states(code, state_transitions: dict):
    starting_point = "A"
    code = starting_point + code
    states = []
    for current, next in zip(code[:-1], code[1:]):
        state_additions = state_transitions[current + next]
        states.append(state_additions)
    return list(itertools.product(*states))


def find_button_combinations_codes(codes, shortest_route: dict):
    result = []
    for code in codes:
        result.extend(find_button_combinations(code, shortest_route))
    return result


states = find_states(code, state_transitions_numeric)
states
# first_bot =

# numeric_bot = find_shortest_route_numeric_code(code)
# first_directional_bot = find_shortest_route_directional_code(numeric_bot)
# second_direction_bot = find_shortest_route_directional_code(first_directional_bot)
# second_direction_bot
# len(second_direction_bot)

[(('<', 1), ('^', 1), ('>', 3), ('v', 3)),
 (('<', 1), ('^', 1), ('^', 3), ('v', 3))]